In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import regularizers
import tensorflow as tf
import os
from tensorflow.keras.callbacks import EarlyStopping
from tqdm.keras import TqdmCallback
import matplotlib.pyplot as plt

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def print_stats(predictions, labels):
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions)
    recall = recall_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    print(f"Accuracy = {accuracy}")
    print(f"Precision = {precision}")
    print(f"Recall = {recall}")
    print(f"F1 score = {f1}")
    return accuracy, precision, recall, f1

In [3]:
data = pd.read_csv("data.csv")

X = data.drop(columns=["Label", "Unnamed: 0"])
y = data["Label"]

In [5]:
mi_scores_series = pd.read_csv("Results_of_feature_selection\mi_scores_series.csv")
selected_features = mi_scores_series["Unnamed: 0"].head(44).astype(str).tolist()
X_selected = X[selected_features]

In [ ]:
X_selected

In [7]:
X_MI = X_selected

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

metrics = []
confusion_matrixs = []
rf = RandomForestClassifier(n_estimators=100, random_state=42)

for n_features in range(1, X_MI.shape[1] + 1):

    rfe = RFE(estimator=rf, n_features_to_select=n_features)
    X_selected = rfe.fit_transform(X_MI, y)

    selected_features = X.columns[rfe.support_]
    print(selected_features)

    kmeans = KMeans(n_clusters=2, n_init=10, random_state=42)
    kmeans.fit(X_selected)
    preds = kmeans.predict(X_selected)
    preds = [1 if pred == 0 else 0 for pred in preds]

    print(f"Number of Features: {n_features}")
    accuracy, precision, recall, f1 = print_stats(preds, y)

    conf_matrix = confusion_matrix(y, preds)
    confusion_matrixs.append({"n_feature": n_features, "conf": conf_matrix})
    print(conf_matrix)

    # Append metrics
    metrics.append({
        "n_features": n_features,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    })


metrics_df = pd.DataFrame(metrics)
metrics_df.to_csv(f"rfe_kmeans.csv", index=False)

plt.figure(figsize=(12, 8))
plt.plot(metrics_df["n_features"], metrics_df["accuracy"], marker="o", label="Accuracy")
plt.plot(
    metrics_df["n_features"], metrics_df["precision"], marker="o", label="Precision"
)
plt.plot(metrics_df["n_features"], metrics_df["recall"], marker="o", label="Recall")
plt.plot(metrics_df["n_features"], metrics_df["f1_score"], marker="o", label="F1 Score")
plt.xlabel("Number of Features", fontsize=20)
plt.legend(fontsize=20)
plt.yticks(fontsize=20)
plt.xticks(fontsize=20)
plt.grid(True)
plt.show()

IndexError: boolean index did not match indexed array along dimension 0; dimension is 359 but corresponding boolean dimension is 44